# Using this Notebook to deploy a YOLOv5 model to Amazon SageMaker Endpoint
---

## Importing Libraries:

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import backend
from sagemaker.tensorflow import TensorFlowModel


## Setting up YOLOv5 model for SageMaker:

### Download YOLOv5 pre-trained models:
- Download any PyTorch models from here: https://github.com/ultralytics/yolov5/releases
- Here we will use `yolov5s.pt` model from: https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt

## Preparing model.tar.gz for deploying to Sagemaker Endpoint
Pretrained YoloV5 models can be accessed through https://github.com/ultralytics/yolov5

First, `git clone https://github.com/ultralytics/yolov5`, then within the `yolov5` directory, you can retrieve a pretrained yolov5 model by running the following command: `python export.py --weights yolov5l.pt --include saved_model --nms`. This will create a directory called `yolov5l_saved_model`. 

In order to deploy this model onto SageMaker Endpoints, it must be packaged in the following structure:

```
export
└──Servo
   └──1
      │  saved_model.pb
      └──assets
      └──variables
         │ variables.data-00000-of-00001
         │ variables.index
```

To get the correct data structure for `model.tar.gz`, copy the contents from `yolov5l_saved_model` into `export/Servo/1/`. Once you have the correct, directory structure for your model, use `tar -czvf model.tar.gz export` to get your `model.tar.gz`.

Upload `model.tar.gz` to your s3 bucket of your choice, and make sure your ec2 instance or sagemaker instance has the correct IAM roles to access the s3 bucket where `model.tar.gz` will be located. 

In my case, my `model.tar.gz` will be uploaded to `s3://sagemaker-us-east-1-171547146718/model.tar.gz`

The following commands can be used in terminal to generate all the necessary `model.tar.gz` object:

```
cd host-yolov5-on-sagemaker-endpoint
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install -r requirements.txt tensorflow-cpu
python export.py --weights yolov5l.pt --include saved_model --nms
mkdir export && mkdir export/Servo
mv yolov5l_saved_model export/Servo/1
tar -czvf model.tar.gz export```

## Define the Model Path and the IAM Role:

In [ ]:
model_data = '<s3://<model.tar.gz path>'
role = '<IAM ROLE>'

## Create a SageMaker TensorFlow Model:

In [ ]:
model = TensorFlowModel(model_data=model_data, 
                        framework_version='2.3', role=role)

## Deploy the model on SageMaker Endpoint:

In [ ]:
INSTANCE_TYPE = 'ml.m5.xlarge'
ENDPOINT_NAME = 'yolov5l-demo'

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         endpoint_name=ENDPOINT_NAME)